#**Машинное обучение ИБ-2024**

#**Домашнее задание 1.**
#Регрессия, KNN, LinearRegression.

В данной домашней работе мы будем строить модели для предсказания цены квартиры в России. Ниже приведено описание некоторых колонок набора данных.

date - дата публикации объявления

price - цена в рублях

level- этаж, на котором находится квартира

levels - количество этажей в квартире

rooms - количество комнат в квартире. Если значение -1, то квартира считается апартаментами.

area - площадь квартиры.

kitchen_area - площадь кухни.

geo_lat - Latitude

geo_lon - Longitude

building_type - материал застройки. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

#Часть 0. Начало работы

Для начала работы с данными импортируем библиотеки, которые понадобятся в данном задании.

In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
import seaborn as sns

Загрузим библиотеку folium для отображения данных на карте по координатам.

In [2]:
!pip install folium

Распакуем наши данные из архива.

In [3]:
!tar -xf Data.zip

да, я виндовод, и что?

Загрузим данные из csv файла в датафрейм.

In [4]:
df = pd.read_csv("input_data.csv", sep=";")

Отобразим на карте координаты наших построек.

In [5]:
import folium
from IPython.display import display

map_df = df.loc[:1000]
print(map_df)
m = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

# Список точек с широтой и долготой
lats = map_df['geo_lat'].loc[:1000]
longs = map_df['geo_lon'].loc[:1000]
# Добавляем точки на карту
for point in zip(lats, longs):
    folium.Marker(
        location=[point[0], point[1]]
    ).add_to(m)

display(m)

            date     price  level  levels  rooms  area  kitchen_area  \
0     2021-01-01   2451300     15      31      1  30.3           0.0   
1     2021-01-01   1450000      5       5      1  33.0           6.0   
2     2021-01-01  10700000      4      13      3  85.0          12.0   
3     2021-01-01   3100000      3       5      3  82.0           9.0   
4     2021-01-01   2500000      2       3      1  30.0           9.0   
...          ...       ...    ...     ...    ...   ...           ...   
996   2021-01-01  12000000      1       9      2  47.3           7.1   
997   2021-01-01   2650000      2       3      2  43.5           6.0   
998   2021-01-01   2880000      5       5      3  62.0           6.0   
999   2021-01-01   3960000      6       6      3  85.0          13.0   
1000  2021-01-01   3170000      6       9      3  56.2        -100.0   

        geo_lat    geo_lon  building_type  object_type  postal_code  \
0     56.780112  60.699355              0            2     62000

# Часть 1. Подготовим данные для обработки моделями машинного обучения.

**0.5 Балл**. География наших наблюдений в наборе данных крайне большая. Однако мы знаем, что стоимость квартир в Москве и Санкт-Петербурге намного выше, чем в среднем по России. Давайте сделаем признаки, который показывают, находится ли квартира в 20 килиметрах от центра Москвы или находится ли квартира в 20 килиметрах от центра Санкт-Петербурга.

Создайте два признака is_Moscow и is_Saint_Peterburg. Для нахождения расстояния по координатам используйте функцию haversine_distance.

Город Москва расположен на 55,7522 северной широты и 37,6156 восточной долготы на высоте 143 метра над уровнем моря.


Географические координаты центра Санкт-Петербурга:
— 59° 57’ северной широты;
— 30° 19’ восточной долготы.
Город расположен у восточной оконечности Финского залива Балтийского моря.

In [6]:
from math import radians, sin, cos, asin, sqrt


def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2.0)**2 + \
        np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance

Moscow_center = (57.7522, 37.6156)
SPB_center = (59.57, 30.19)


df['is_Moscow'] = (haversine_distance(Moscow_center[0], Moscow_center[1], df['geo_lat'], df['geo_lon']) <= 20).astype(bool)
df['is_SPB'] = (haversine_distance(SPB_center[0], SPB_center[1], df['geo_lat'], df['geo_lon']) <= 20).astype(bool)


**0.5 Балла**. В нашем наборе данных есть признаки, которые мы теоретически можем использовать, например postal_code, но мы это будем делать в рамках домашней работы очень-очень долго. Поэтому предлагается удалить ненужные признаки из датафрейма.

Удалим geo_lat,	geo_lon,	object_type,	postal_code,	street_id,	id_region,	house_id.

In [7]:
df.drop(columns=["geo_lat", "geo_lon", "object_type", "postal_code", "street_id", "id_region", "house_id"], inplace=True)


In [8]:
print(df)


                date     price  level  levels  rooms  area  kitchen_area  \
0         2021-01-01   2451300     15      31      1  30.3           0.0   
1         2021-01-01   1450000      5       5      1  33.0           6.0   
2         2021-01-01  10700000      4      13      3  85.0          12.0   
3         2021-01-01   3100000      3       5      3  82.0           9.0   
4         2021-01-01   2500000      2       3      1  30.0           9.0   
...              ...       ...    ...     ...    ...   ...           ...   
11358145  2021-12-31   6099000      4       9      3  65.0           0.0   
11358146  2021-12-31   2490000      1      10      2  56.9           0.0   
11358147  2021-12-31    850000      2       2      2  37.0           5.0   
11358148  2021-12-31   4360000      5       5      1  36.0           9.0   
11358149  2021-12-31   1850000      1       5      1  33.0           7.0   

          building_type  is_Moscow  is_SPB  
0                     0      False   False

**0.5 Балл**. Для начала Вам предлагается проанализировать Ваши оставшиеся признаки (колонки) в наборе данных. Какие колонки категориальные? Какие числовые?

Категориальные: building_type, is_Moscow, is_SPB

предполагаю, что is_Moscow и is_SPB энкодить не надо, т.к. они уже 1/0


Числовые: price, level, levels, rooms, area, kitchen_area, date(после обработки)

Давайте закодируем категориальные признаки с помощью OneHot-Encoding. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [9]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first', sparse_output=False)
encoded = encoder.fit_transform(df[['building_type']])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['building_type']), index=df.index)
df = pd.concat([df.drop('building_type', axis=1), encoded_df], axis=1)
print(df)

                date     price  level  levels  rooms  area  kitchen_area  \
0         2021-01-01   2451300     15      31      1  30.3           0.0   
1         2021-01-01   1450000      5       5      1  33.0           6.0   
2         2021-01-01  10700000      4      13      3  85.0          12.0   
3         2021-01-01   3100000      3       5      3  82.0           9.0   
4         2021-01-01   2500000      2       3      1  30.0           9.0   
...              ...       ...    ...     ...    ...   ...           ...   
11358145  2021-12-31   6099000      4       9      3  65.0           0.0   
11358146  2021-12-31   2490000      1      10      2  56.9           0.0   
11358147  2021-12-31    850000      2       2      2  37.0           5.0   
11358148  2021-12-31   4360000      5       5      1  36.0           9.0   
11358149  2021-12-31   1850000      1       5      1  33.0           7.0   

          is_Moscow  is_SPB  building_type_1  building_type_2  \
0             False   

**0.5 Балл**. Поработаем с числовыми признаками:


1.   Добавьте в ваш датасет два признака: количество дней со дня первого наблюдения (разница между датами объявлений). Возможно, для предсказания цены не так важен этаж, как важно отношение этажа квартиры на количество этажей в доме, добавьте этот признак. После добавления нового признака колонку date можно удалить.

Ну я снесу тогда наверное все части таблицы где нету floor ratio

2.   Числовые признаки могут иметь разные порядки. Давайте отнормируем числовые признаки с помощью StandartScaller https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.



In [10]:


df['date'] = pd.to_datetime(df['date'], errors='coerce')
first_date = df['date'].min()
df['days_since_first_observation'] = (df['date'] - first_date).dt.days
df.drop('date', axis = 1, inplace = True)

df['floor_ratio'] = df['level'] / df['levels']
df = df[~df['floor_ratio'].isin([float('inf'), -float('inf')])]
df = df.dropna() 


from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[df.columns.tolist()] = scaler.fit_transform(df[df.columns.tolist()])
print(df)

             price     level    levels     rooms      area  kitchen_area  \
0        -0.021928  1.622845  2.665137 -0.621436 -0.840594      0.082515   
1        -0.026992 -0.270032 -0.937314 -0.621436 -0.741061      0.267578   
2         0.019788 -0.459320  0.171132  1.106218  1.175876      0.452641   
3        -0.018648 -0.648608 -0.937314  1.106218  1.065284      0.360110   
4        -0.021682 -0.837895 -1.214426 -0.621436 -0.851654      0.360110   
...            ...       ...       ...       ...       ...           ...   
11358145 -0.003481 -0.459320 -0.383091  1.106218  0.438593      0.082515   
11358146 -0.021733 -1.027183 -0.244535  0.242391  0.139993      0.082515   
11358147 -0.030027 -0.837895 -1.352982  0.242391 -0.593604      0.236734   
11358148 -0.012275 -0.270032 -0.937314 -0.621436 -0.630469      0.360110   
11358149 -0.024969 -1.027183 -0.937314 -0.621436 -0.741061      0.298422   

          is_Moscow    is_SPB  building_type_1  building_type_2  \
0          -0.00089 

**2 Балла**. Реализуйте класс KNNRegressor, который должен делать регрессию методом k ближайших соседей.

In [11]:
from sklearn.metrics import pairwise_distances

class KNNRegressor:
    def __init__(self, n_neighbors=5, metric='euclidean'):
      self.n_neighbors = n_neighbors
      self.metric = metric

    def fit(self, X, y):
      self.X_train = np.array(X)
      self.y_train = np.array(y)
      if self.X_train.shape[0] != self.y_train.shape[0]:
         raise ValueError()

    def predict(self, X):
        X = np.array(X)
        distances = pairwise_distances(X, self.X_train, metric=self.metric)
        return np.mean(self.y_train[np.argsort(distances, axis=1)[:, :self.n_neighbors]], axis=1)

**3 Балла**. Реализуйте класс LinearRegression, поддерживающий обучение градиентными спусками SGD, Momentum, AdaGrad. Используйте градиент для оптимизации функции потерь MSE.

In [12]:
class LinearRegression:
    def __init__(self, learning_rate=0.01, optimization='SGD', epsilon=1e-8, decay_rate=0.9, max_iter=1000):
        self.learning_rate = learning_rate
        self.optimization = optimization
        self.epsilon = epsilon
        self.decay_rate = decay_rate
        self.max_iter = max_iter
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        if self.optimization == 'Momentum':
            velocity_w = np.zeros(n_features)
            velocity_b = 0
        elif self.optimization == 'AdaGrad':
            cache_w = np.zeros(n_features)
            cache_b = 0
        
        for i in range(self.max_iter):
            y_predicted = np.dot(X, self.weights) + self.bias
            error = y_predicted - y
            dw = (2 / n_samples) * np.dot(X.T, error)
            db = (2 / n_samples) * np.sum(error)
            if self.optimization == 'SGD':
                self.weights -= self.learning_rate * dw
                self.bias -= self.learning_rate * db
            
            elif self.optimization == 'Momentum':
                velocity_w = self.decay_rate * velocity_w + self.learning_rate * dw
                velocity_b = self.decay_rate * velocity_b + self.learning_rate * db
                self.weights -= velocity_w
                self.bias -= velocity_b
            
            else:
                cache_w += dw ** 2
                cache_b += db ** 2
                self.weights -= (self.learning_rate / (np.sqrt(cache_w) + self.epsilon)) * dw
                self.bias -= (self.learning_rate / (np.sqrt(cache_b) + self.epsilon)) * db

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

# Часть 2. Эксперименты с моделями машинного обучения.

**3 Балла**. Проведите эксперименты с написанными Вами методами машинного обучения. Выделите обучающую и тестовую выборки в отношении 0,8 и 0,2 соответственно. Измерьте ошибку MSE, MAE, RMSE. Заиспользуйте методы KNNRegressor и LinearRegression из библиотеки sklearn, сравните качество Ваших решений и библиотечных.

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor as SKKNeighborsRegressor
from sklearn.linear_model import LinearRegression as SKLinearRegression

df = df.dropna()
df = df.head(10000)
X = df.drop('price', axis=1).values
y = df['price'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=131313)


custom_knn = KNNRegressor(n_neighbors=5, metric='euclidean')
custom_knn.fit(X_train, y_train)
y_pred_custom_knn = custom_knn.predict(X_test)

custom_lr = LinearRegression(learning_rate=0.01, optimization='SGD', max_iter=1000)
custom_lr.fit(X_train, y_train)
y_pred_custom_lr = custom_lr.predict(X_test)

sklearn_knn = SKKNeighborsRegressor(n_neighbors=5, metric='euclidean')
sklearn_knn.fit(X_train, y_train)
y_pred_sklearn_knn = sklearn_knn.predict(X_test)

sklearn_lr = SKLinearRegression()
sklearn_lr.fit(X_train, y_train)
y_pred_sklearn_lr = sklearn_lr.predict(X_test)

def compute_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return mse, mae, rmse

print(compute_metrics(y_test, y_pred_custom_knn))

print(compute_metrics(y_test, y_pred_custom_lr))

print(compute_metrics(y_test, y_pred_sklearn_knn))

print(compute_metrics(y_test, y_pred_sklearn_lr))


(0.014776337177756333, 0.017670474633907612, 0.12155795810129559)
(0.005068691055954395, 0.021398350873437395, 0.07119474036720967)
(0.014776370478973783, 0.01767236445925053, 0.12155809507792471)
(0.005069712943611961, 0.021483160315155875, 0.07120191671304896)
